# La théorie des jeux

Auteur : Philippe Mathieu, CRISTAL Lab, SMAC Team, University of Lille, email : philippe.mathieu@univ-lille.fr

Contributeurs : Louisa Fodil (CRISTAL/SMAC), Céline Petitpré (CRISTAL/SMAC)

Creation : 18/01/2018



Céline : Cette feuille  parle uniquement de théorie des jeux simulanés, et d'équilibre. Onpeut parler du dilemme du prisonnier mais on ne parle surtout pas du dilemme itéré du prisonnier. On ne parle pas ici de jeux itérés ! par contre on peut s'amuser à montrer les caractérisques d'un jeu particulier comme le dilemme du prisonnier


## Introduction

La théorie des jeux est la discipline qui étudie de manière formelle les interactions que peuvent avoir plusieurs individus dans une situation conflictuelle. Le terme "jeu" n'a a priori rien de ludique, Von Neumann l'ayant par exemple développée pour la modélisation des relations Franco-Américaines de 1945 et notamment lors de l'épisode de la baie des cochons. La théorie des jeux décompose les jeux en fonction du nombre de joueurs (généralement 2), puis principalement en deux grandes familles : les jeux simultanés, dans lesquels les joueurs jouent simultanément et donc ne savent pas en jouant ce que joue l'adversaire (papier-feuiile-ciseaux par exemple) et les jeux non simultanés où chacun joue tour à tour (les échecs par exemple). 

Nous nous plaçons ici dans le cadre des **jeux à deux joueurs, simultanés**. On représente en général ce type de jeu par une matrice de gains. Les coups d'un des deux joueurs se trouvent en abscisse et ceux de l'autre en ordonnée. A l'intersection de chaque case, correspondant à une situation, on note les points qui seront distribués à chacun. La théorie décompose encore ces deux en deux autres familles, les jeux à somme nulle dans lesquels tout se que perd l'un est gagné par l'autre, et les jeux à somme non nulle dans lesquels les points distribués sont différents selon les coups joués. Nous sommes ici dans un jeu **à somme non nulle**.

Cette feuille a pour objectif de montrer comment établir et comparer des stratégies à ce jeu.
On trouvera plus de détails sur [Wikipedia](http://www.wikidedia.org)



# Une matrice de gains

Tout d'abord, il est necessaire de pouvoir coder une matrice de gains. Une classe `Game` va nous
permettre de stocker les gains des deux joueurs pour une situation donnée. 
Un objet Game prend un tableau de couples en paramètre, correspondant aux scores de chaque issue, ainsi que le tableau des noms des actions correspondantes.
En interne, Le package *numpy* offre un objet `Array` idéal pour stocker et manipuler cela. Pour un jeu à `n` coups, nous créons un Array `n*n` de couples de valeurs `(x,y)` avec `x` le gain du joueur 1 et `y` le gain du joueur 2. 

Cette classe doit permettre aussi de fournir le ou les équilibres potentiels.
Pour calculer la situation de "non regret" correspondant à l'équilibre de Nash il suffit de noter les réponses du joueur1 les plus adaptés à chaque stratégie du joueur 2 (donc calculer les max(x) dans la chaque colonne), puis calculer les meilleures réponses du joueur 2 aux stratégies du joueur 1 (donc calculer les max(y) dans chaque ligne). Si une issue possede 2 max, c'est un équilibre de Nash. Selon les jeux, il peut bien sûr y en avoir 1, plusieurs ou pas du tout. 
L'usage d'un `np.Array` facilite énormément les choses puisqu'il est possible d'avoir les vecteurs de valeurs max en ligne ou en colonne. Il suffit donc de fabriquer des matrices booléennes dans chaque cas et d'en faire un `ou` logique.

In [3]:
import numpy as np
import math

class Game:
    def __init__(self, tab, actions):
        self.actions=actions
        m=np.array(tab,dtype=[('x', 'int32'), ('y', 'int32')])
        self.size = int(math.sqrt(len(tab)))
        self.scores=m.reshape(self.size,self.size)

In [4]:
dp =[(3,3),(0,5),(5,0),(1,1)]   # Dilemme du prisonnier : 1 equilibrium
gs=[(3,2),(1,1),(0,0),(2,3)]     # Guerre des sexes : 2 equilibria
mp=[(1,-1),(-1,1),(-1,1),(1,-1)] # matching pennies : 0 equilibrium
rpc=[(0,0),(-1,1),(1,-1),(1,-1),(0,0),(-1,1),(-1,1),(1,-1),(0,0)] # papier feuille ciseaux  : 0 equilibrium
g = Game(dp,['C','D'])
g.scores

array([[(3, 3), (0, 5)],
       [(5, 0), (1, 1)]], dtype=[('x', '<i4'), ('y', '<i4')])

# La notion d'équilibre

on explique ici l'hypothèse de rationalité des agents en économie, et la recherche d'équilibre.

Les économistes se sont très vite emparés de ces modèles pour étudier différents problèmes économiques. Afin d'identifier la meilleure manière de jouer à un jeu fixé, ils se sont basés sur la notion abstraite de rationalité du joueur. Partant du principe que, dans le pire des cas, l'adversaire est aussi rationnel, le raisonnement que je m'applique doit donc aussi être appliqué à mon adversaire. très vite, deux définitions de la rationalité sont apparues, définissant ainsi un (ou éventuellement plusieurs) point fixe d'un jeu, point vers lequel rationnellement, si tout le monde réfléchis de manière identique, on parviendra :
1. *L'équilibre de Nash*, du nom de son inventeur, John Nash. L'équilibre de Nash est une situation de non regret de chacun des joueurs. *"nous avons joué simultanément, mais maintenant que je sais ce que l'autre a joué, je n'ai aucun regret"*
2. *L'optimum de Pareto*, du nom de son inventeur, Wilfried Pareto. l'optimum de Pareto est une situation où aucune situation n'est supérieure pour les deux joueurs, et donc ils n'ont aucun interêt collectivement à changer.

Plusieurs notions co-existent dans la littérature. 
- optimum de pareto :
- équilibre de Nash : situation de non regret
- élimination des stratégies dominées

In [5]:
import numpy as np
import math

class Game:
    def __init__(self, tab, actions):
        self.actions=actions
        m=np.array(tab,dtype=[('x', 'int32'), ('y', 'int32')])
        self.size = int(math.sqrt(len(tab)))
        self.scores=m.reshape(self.size,self.size)

    def nash(self):
        max_x = np.matrix(self.scores['x'].max(0)).repeat(self.size, axis=0)
        bool_x = self.scores['x'] == max_x
        max_y = np.matrix(self.scores['y'].max(1)).transpose().repeat(self.size, axis=1)
        bool_y = self.scores['y'] == max_y
        bool_x_y = bool_x & bool_y
        return self.scores[bool_x_y]

    def isPareto(self, t, s):
        return True if (len(s)==0) else (s[0][0] <= t[0] or s[0][1] <= t[1]) and self.isPareto(t, s[1:])

    def pareto(self):
        res = []
        for s in self.scores:
            if(self.isPareto(s,self.scores)):
                res.append(s) 
        return res

In [7]:
faire ici qq experiences sur des jeux différents du DP, genre gs ou mp ou un autre au choix mais connu dans la litterature

SyntaxError: invalid syntax (<ipython-input-7-f8aab8e1e409>, line 1)

## Le dilemme du prisonnier

Le Dilemme du prisonnier, identifié par M. Flood and M. Dresher de la Rand Corporation en 1950, est un modèle de théorie des jeux spécialement créé pour montrer que l'équilibre de Nash n'est pas toujours une bonne idée.


Celine : on s'autorise dans cette feuille à parler du dilemme du prisonnier, mais pas du dilemme itéré du prisonnier. On ne parle pas ici de jeux itérés ! par contre on peut s'amuser à montrer les caractérisqieus d'un jeu particulier comme le dilemme du prisonnier (que j'ai ici appelé dp). Faire qq expleriences sur le dp juste en dessous : vérifier Nash, pareto et elimination des dominées


In [6]:
g = Game(dip,['C','D'])
g.scores
g.nash()
#g.pareto()

NameError: name 'dip' is not defined

## Amusons nous

### Créer une matrice de jeu au hasard

In [4]:
x = np.random.randint(0, 5, (3,3))
y = np.random.randint(0, 5, (3,3))
couples = [(a,b) for a,b in zip(x.flatten(),y.flatten())]
g = Game(couples, None)
print(g.scores)
print(g.nash())

[[(1, 3) (0, 3) (1, 2)]
 [(4, 1) (4, 4) (1, 3)]
 [(3, 1) (2, 4) (0, 3)]]
[(4, 4)]


### Générer toutes les matrices

En Python il est facile d'énumérer tous les jeux à 2 coups à partir de valeurs fixées. Par exemple tous jeux que l'on peut construire avec les valeurs 1 et 2.
La Librairie `itertools` fournit de nombreux itérateurs efficaces, notamment pour les combinaisons, permutations et produits cartésiens. Ici c'est le produit cartésien des valeurs qui nous interesse.
On peut ensuite par exemple compter combien de jeux ont 0,1 ou plusieurs équilibres de Nash.


In [5]:
import itertools;
import random;

def numberOfGames(valeurs, nbCoups):
    return len(valeurs)**((nbCoups**2)*2)

print(numberOfGames([1,2],2))

def enumAllGames(valeurs, nbCoups):
    res = [q for q in itertools.product([p for p in itertools.product(list(valeurs), repeat=2)], repeat=nbCoups**2)]
    return [[res[j][k] for k in range(nbCoups**2)] for j in range(len(res))]

n = enumAllGames([1,2],2)
print("Impression de 10 jeux aléatoires trouvés sur "+str(numberOfGames([1,2],2)))
for i in range (10):
    print(random.choice(n))
        
def countNashEquilibria(valeurs, coups):
    results = [Game(i,None).nash().size for i in enumAllGames(valeurs, coups)]
    return dict((i,results.count(i)) for i in set(results))

# Combien de jeux à 2 coups batis sur (1,2) on x equilbres de Nash
countNashEquilibria([1,2],2)

256
Impression de 10 jeux aléatoires trouvés sur 256
[(2, 1), (2, 1), (1, 1), (1, 2)]
[(2, 2), (2, 1), (2, 2), (1, 1)]
[(2, 2), (2, 1), (2, 1), (1, 2)]
[(2, 2), (1, 1), (1, 2), (1, 1)]
[(2, 2), (1, 1), (1, 1), (2, 2)]
[(1, 1), (1, 1), (1, 1), (2, 1)]
[(2, 1), (1, 2), (2, 2), (1, 1)]
[(2, 1), (2, 2), (2, 1), (2, 1)]
[(1, 2), (2, 1), (2, 2), (1, 2)]
[(1, 1), (2, 2), (1, 1), (1, 2)]


{0: 2, 1: 44, 2: 114, 3: 80, 4: 16}

# Trouver des jeux avec des contraintes particulières
Cherchons par exemple, les jeux à deux coups dont les valeurs sont prises dans (1,2) et qui ont exactement les mêmes equilibres nash et pareto

In [6]:
nbCoups=2
res = [q for q in itertools.combinations([p for p in itertools.product([0,1,3,5], repeat=2)], nbCoups**2)]
games = [[res[j][k] for k in range(nbCoups**2)] for j in range(len(res))]
print(str(len(games))+" jeux étudiés.")
r = []
for g in games:
    if ((sorted(Game(g,None).pareto()) == sorted(Game(g,None).nash().tolist())) and (len(set(g)) == len(g))):
        r.append(g)
r


1820 jeux étudiés.


TypeError: '>=' not supported between instances of 'numpy.ndarray' and 'numpy.ndarray'

# Bibliographie

A refaire avec uniquelment de la théorie des jeux

- Von Neumann et Morgenstern etc ....